In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd
import torch.utils.data
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Dataset

In [3]:
train_dataset = dataset.CIFAR100(root="./CIFAR100/train", train=True, transform=None, download=True)

100%|██████████| 169M/169M [00:06<00:00, 24.9MB/s]


Extracting ./CIFAR100/train/cifar-100-python.tar.gz to ./CIFAR100/train


In [4]:
x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
mean = np.mean(x, axis=(0, 1))/255
std = np.std(x, axis=(0,1))/255

mean = mean.tolist()
std = std.tolist()

print("mean:", mean)
print("std:", std)

mean: [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std: [0.26733428587941854, 0.25643846292120615, 0.2761504713263903]


In [5]:
transform = T.Compose([T.ToTensor(),
                       T.Normalize(mean, std, inplace=True)])
train_dataset = dataset.CIFAR100(root="./CIFAR100/train", train=True, transform=transform, download=True)
valid_dataset = dataset.CIFAR100(root="./CIFAR100/val", train=True, transform=transform, download=True)
test_dataset = dataset.CIFAR100(root="./CIFAR100/test", train=False, transform=transform, download=True)

Files already downloaded and verified


100%|██████████| 169M/169M [00:09<00:00, 17.7MB/s]


Extracting ./CIFAR100/val/cifar-100-python.tar.gz to ./CIFAR100/val


100%|██████████| 169M/169M [00:06<00:00, 24.9MB/s]


Extracting ./CIFAR100/test/cifar-100-python.tar.gz to ./CIFAR100/test


In [6]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [7]:
images, labels = next(iter(train_loader))

In [8]:
print(images.shape)
print(labels.shape)
print(labels)

torch.Size([256, 3, 32, 32])
torch.Size([256])
tensor([27, 10, 73, 51, 53, 82, 21, 44, 47,  5, 94, 57, 17, 94, 92, 91, 53, 76,
        45, 46, 52, 30, 30, 85, 39, 70,  2,  6, 30, 58, 36, 67, 61, 38, 28, 10,
        61, 46, 43, 61, 27, 22, 86, 11, 43, 87, 78, 47, 97,  2, 94, 16, 79, 92,
        84,  8, 91, 35, 61, 55, 75, 42, 60, 49, 41, 96, 46, 97, 37, 34, 58,  4,
        89, 51, 92, 62, 66, 40, 95, 26, 31, 86, 33, 82, 58, 14, 68, 26, 96, 23,
        87, 82, 15, 65, 26,  3,  1, 92, 60, 32, 16, 44, 20, 86, 36, 56, 99, 61,
        72, 46, 91, 84, 89, 30, 37,  0, 69, 68, 58, 11, 46, 41, 98, 46, 15,  2,
        21, 37, 59, 79, 88, 71, 65, 70, 60, 21, 62, 92, 40, 29,  6, 48, 40, 66,
        90, 98, 44, 64, 46, 13,  6,  6, 18, 80, 33, 76, 27, 45, 11, 61, 59, 96,
        60, 25, 80, 66, 48, 87, 24,  1, 35, 56,  2, 30,  3,  7, 29, 47, 81, 23,
         2, 91, 12,  3, 40, 15, 70, 68, 25,  3, 31, 73, 10, 87, 85, 45, 74, 58,
         7, 56, 20, 84, 93, 34, 15, 45,  4, 82, 61, 42, 10, 82, 91, 46, 5

## Model

In [9]:
# No CBAM
import torch
import torch.nn as nn

class InceptionModule(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(InceptionModule, self).__init__()

        # 1x1 convolution branch
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, ch1x1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(ch1x1),
            nn.ReLU(inplace=True)
        )

        # 3x3 convolution branch
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(ch3x3red),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(ch3x3),
            nn.ReLU(inplace=True)
        )

        # 5x5 convolution branch
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(ch5x5red),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(ch5x5),
            nn.ReLU(inplace=True)
        )

        # Max pooling branch
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(pool_proj),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return torch.cat([
            self.branch1(x),
            self.branch2(x),
            self.branch3(x),
            self.branch4(x)
        ], 1)

class AuxiliaryClassifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(AuxiliaryClassifier, self).__init__()
        self.averagePool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.averagePool(x)
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class GoogleNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(GoogleNet, self).__init__()

        # Initial convolution layers
        self.prelayers = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            nn.Conv2d(64, 64, kernel_size=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)
        )

        # Inception modules
        self.inception3a = InceptionModule(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)

        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)

        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)

        # Auxiliary Classifiers
        self.aux1 = AuxiliaryClassifier(512, num_classes)
        self.aux2 = AuxiliaryClassifier(528, num_classes)

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        # Initial layers
        x = self.prelayers(x)

        # Inception modules
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        aux1 = self.aux1(x) if self.training else None

        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        aux2 = self.aux2(x) if self.training else None

        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        # Final classification
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        return x, aux1, aux2 if self.training else x

In [10]:
# With CBAM
import torch
import torch.nn as nn
import torch.nn.functional as F

class SAM(nn.Module):
    def __init__(self, bias=False):
        super(SAM, self).__init__()
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3, dilation=1, bias=self.bias)

    def forward(self, x):
        max_pool = torch.max(x, 1)[0].unsqueeze(1)
        avg_pool = torch.mean(x, 1).unsqueeze(1)
        concat = torch.cat((max_pool, avg_pool), dim=1)
        output = self.conv(concat)
        output = output * x
        return output

class CAM(nn.Module):
    def __init__(self, channels, r):
        super(CAM, self).__init__()
        self.channels = channels
        self.r = r
        self.linear = nn.Sequential(
            nn.Linear(in_features=self.channels, out_features=max(self.channels//self.r, 1), bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=max(self.channels//self.r, 1), out_features=self.channels, bias=True))

    def forward(self, x):
        max_pool = F.adaptive_max_pool2d(x, output_size=1)
        avg_pool = F.adaptive_avg_pool2d(x, output_size=1)
        b, c, _, _ = x.size()
        linear_max = self.linear(max_pool.view(b, c)).view(b, c, 1, 1)
        linear_avg = self.linear(avg_pool.view(b, c)).view(b, c, 1, 1)
        output = linear_max + linear_avg
        output = F.sigmoid(output) * x
        return output

class CBAM(nn.Module):
    def __init__(self, channels, r):
        super(CBAM, self).__init__()
        self.channels = channels
        self.r = r
        self.sam = SAM(bias=False)
        self.cam = CAM(channels=self.channels, r=self.r)

    def forward(self, x):
        output = self.cam(x)
        output = self.sam(output)
        return output + x

class InceptionModule(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj, use_cbam=False, r=4):
        super(InceptionModule, self).__init__()

        # 1x1 convolution branch
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, ch1x1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(ch1x1),
            nn.ReLU(inplace=True)
        )

        # 3x3 convolution branch
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(ch3x3red),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(ch3x3),
            nn.ReLU(inplace=True)
        )

        # 5x5 convolution branch
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(ch5x5red),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(ch5x5),
            nn.ReLU(inplace=True)
        )

        # Max pooling branch
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(pool_proj),
            nn.ReLU(inplace=True)
        )

        # Optional CBAM
        self.cbam = CBAM(ch1x1 + ch3x3 + ch5x5 + pool_proj, r) if use_cbam else None

    def forward(self, x):
        outputs = [
            self.branch1(x),
            self.branch2(x),
            self.branch3(x),
            self.branch4(x)
        ]

        x = torch.cat(outputs, 1)

        if self.cbam is not None:
            x = self.cbam(x)

        return x

class AuxiliaryClassifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(AuxiliaryClassifier, self).__init__()
        self.averagePool = nn.AdaptiveAvgPool2d(output_size=(4, 4))
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.averagePool(x)
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class GoogleNet_CBAM(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000, r=4):
        super(GoogleNet_CBAM, self).__init__()

        # Initial convolution layers
        self.prelayers = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            nn.Conv2d(64, 64, kernel_size=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)
        )

        # Inception modules with optional CBAM
        self.inception3a = InceptionModule(192, 64, 96, 128, 16, 32, 32, use_cbam=True, r=r)
        self.inception3b = InceptionModule(256, 128, 128, 192, 32, 96, 64, use_cbam=True, r=r)

        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception4a = InceptionModule(480, 192, 96, 208, 16, 48, 64, use_cbam=True, r=r)
        self.inception4b = InceptionModule(512, 160, 112, 224, 24, 64, 64, use_cbam=True, r=r)
        self.inception4c = InceptionModule(512, 128, 128, 256, 24, 64, 64, use_cbam=True, r=r)
        self.inception4d = InceptionModule(512, 112, 144, 288, 32, 64, 64, use_cbam=True, r=r)
        self.inception4e = InceptionModule(528, 256, 160, 320, 32, 128, 128, use_cbam=True, r=r)

        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception5a = InceptionModule(832, 256, 160, 320, 32, 128, 128, use_cbam=True, r=r)
        self.inception5b = InceptionModule(832, 384, 192, 384, 48, 128, 128, use_cbam=True, r=r)

        # Auxiliary Classifiers
        self.aux1 = AuxiliaryClassifier(512, num_classes)
        self.aux2 = AuxiliaryClassifier(528, num_classes)

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        # Initial layers
        x = self.prelayers(x)

        # Inception modules
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        aux1 = self.aux1(x) if self.training else None

        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        aux2 = self.aux2(x) if self.training else None

        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        # Final classification
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        return x, aux1, aux2 if self.training else x

## Train

In [11]:
model1 = GoogleNet_CBAM(in_channels=3, num_classes=100)
optimizer1 = optim.AdamW(model1.parameters(), lr=0.0001, weight_decay=0.005)
criterion1 = nn.CrossEntropyLoss()
total_epochs = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model1 = model1.to(device)

In [12]:
model2 = GoogleNet(in_channels=3, num_classes=100)
optimizer2 = optim.AdamW(model2.parameters(), lr=0.0001, weight_decay=0.005)
criterion2 = nn.CrossEntropyLoss()
total_epochs = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model2 = model2.to(device)

In [13]:
def train(model, optimizer, train_loader, val_loader, criterion, total_epochs, name):
    print("Training Begin!")
    print()
    best_accuracy = 0

    for epoch in range(total_epochs):
        model.train()

        for step, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            # Get outputs from the model
            outputs = model(images)

            # Handle tuple output
            if isinstance(outputs, tuple):
                main_output, aux1, aux2 = outputs
                loss = criterion(main_output, labels)

                # Include auxiliary loss (optional)
                if aux1 is not None and aux2 is not None:
                    loss_aux1 = criterion(aux1, labels)
                    loss_aux2 = criterion(aux2, labels)
                    loss += 0.3 * (loss_aux1 + loss_aux2)  # Weighted auxiliary losses

            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            if (step+1) % 30 == 0:
                print('Epoch: [{}/{}] | Step: [{}/{}] | Loss: {:.4f}'.format(epoch+1, total_epochs, step+1, len(train_loader), loss.item()))

        # Validation
        with torch.no_grad():
            print("Validating...")
            model.eval()
            total = 0
            correct = 0
            for (images, labels) in val_loader:
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

                outputs = model(images)
                if isinstance(outputs, tuple):  # Get main output for validation
                    outputs = outputs[0]

                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            acc = (correct/total)*100
            if acc > best_accuracy:
                best_accuracy = acc
                torch.save(model.state_dict(), f"./{name}.pt")
            print(f"Current Accuracy: {acc:.3f}%")
            print(f"Best Accuracy: {best_accuracy:.3f}%")
            print()

    print('Train Finished!')

In [14]:
train(model=model1, optimizer=optimizer1, train_loader=train_loader, val_loader=valid_loader, criterion=criterion1, total_epochs=total_epochs,name="model_cbam")

Training Begin!

Epoch: [1/50] | Step: [30/196] | Loss: 7.3742
Epoch: [1/50] | Step: [60/196] | Loss: 7.2986
Epoch: [1/50] | Step: [90/196] | Loss: 6.8892
Epoch: [1/50] | Step: [120/196] | Loss: 6.7202
Epoch: [1/50] | Step: [150/196] | Loss: 6.3780
Epoch: [1/50] | Step: [180/196] | Loss: 6.3991
Validating...
Current Accuracy: 12.696%
Best Accuracy: 12.696%

Epoch: [2/50] | Step: [30/196] | Loss: 6.0709
Epoch: [2/50] | Step: [60/196] | Loss: 6.0287
Epoch: [2/50] | Step: [90/196] | Loss: 5.9205
Epoch: [2/50] | Step: [120/196] | Loss: 5.8411
Epoch: [2/50] | Step: [150/196] | Loss: 5.6781
Epoch: [2/50] | Step: [180/196] | Loss: 5.8708
Validating...
Current Accuracy: 21.348%
Best Accuracy: 21.348%

Epoch: [3/50] | Step: [30/196] | Loss: 5.4247
Epoch: [3/50] | Step: [60/196] | Loss: 5.2067
Epoch: [3/50] | Step: [90/196] | Loss: 5.2624
Epoch: [3/50] | Step: [120/196] | Loss: 5.2374
Epoch: [3/50] | Step: [150/196] | Loss: 5.1313
Epoch: [3/50] | Step: [180/196] | Loss: 5.0951
Validating...
Curr

In [15]:
train(model=model2, optimizer=optimizer2, train_loader=train_loader, val_loader=valid_loader, criterion=criterion2, total_epochs=total_epochs,name="model_without_cbam")

Training Begin!

Epoch: [1/50] | Step: [30/196] | Loss: 7.3109
Epoch: [1/50] | Step: [60/196] | Loss: 6.9753
Epoch: [1/50] | Step: [90/196] | Loss: 6.8671
Epoch: [1/50] | Step: [120/196] | Loss: 6.6637
Epoch: [1/50] | Step: [150/196] | Loss: 6.6039
Epoch: [1/50] | Step: [180/196] | Loss: 6.3111
Validating...
Current Accuracy: 12.968%
Best Accuracy: 12.968%

Epoch: [2/50] | Step: [30/196] | Loss: 6.2543
Epoch: [2/50] | Step: [60/196] | Loss: 5.9375
Epoch: [2/50] | Step: [90/196] | Loss: 5.8822
Epoch: [2/50] | Step: [120/196] | Loss: 5.9700
Epoch: [2/50] | Step: [150/196] | Loss: 5.8076
Epoch: [2/50] | Step: [180/196] | Loss: 5.5451
Validating...
Current Accuracy: 21.000%
Best Accuracy: 21.000%

Epoch: [3/50] | Step: [30/196] | Loss: 5.3566
Epoch: [3/50] | Step: [60/196] | Loss: 5.2868
Epoch: [3/50] | Step: [90/196] | Loss: 5.5324
Epoch: [3/50] | Step: [120/196] | Loss: 5.4339
Epoch: [3/50] | Step: [150/196] | Loss: 5.2963
Epoch: [3/50] | Step: [180/196] | Loss: 5.0945
Validating...
Curr

In [16]:
def test(model, test_loader, criterion):
    print("Evaluating Test Dataset...")
    print()
    best_accuracy = 0

    with torch.no_grad():
        model.eval()
        total = 0
        correct = 0
        count = 0

        for (images, labels) in test_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(images)

            # Handle tuple outputs (e.g., for GoogleNet with auxiliary classifiers)
            if isinstance(outputs, tuple):
                outputs = outputs[0]  # Only take the primary output

            # top-1 accuracy
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += torch.eq(predicted, labels).sum().item()

            # top-5 accuracy
            _, predicted = torch.topk(outputs, 5)
            for gt, pred in zip(labels, predicted):
                if gt in pred:
                    count += 1

        top1_acc = (correct / total) * 100
        top5_acc = (count / total) * 100
        print(f"Test Top-1 Accuracy: {top1_acc:.2f}%")
        print(f"Test Top-5 Accuracy: {top5_acc:.2f}%")

In [17]:
model = GoogleNet(in_channels=3, num_classes=100).cuda()
model.load_state_dict(torch.load("./model_without_cbam.pt"))
test(model=model, test_loader=test_loader, criterion=criterion1) # No CBAM

<ipython-input-17-602af871309b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./model_without_cbam.pt"))


Evaluating Test Dataset...

Test Top-1 Accuracy: 28.07%
Test Top-5 Accuracy: 54.61%


In [18]:
model = GoogleNet_CBAM(in_channels=3, num_classes=100).cuda()
model.load_state_dict(torch.load("./model_cbam.pt"))
test(model=model, test_loader=test_loader, criterion=criterion2) # With CBAM

Evaluating Test Dataset...



<ipython-input-18-58e10ff0456e>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./model_cbam.pt"))


Test Top-1 Accuracy: 28.92%
Test Top-5 Accuracy: 55.46%
